In [2]:
%matplotlib agg
from river.datasets import synth
from clustream import CluStream
from river import drift
import math
import numpy as np
import matplotlib.pyplot as plt
from river.utils import dict2numpy
from scipy.spatial import distance_matrix
from time import time
from river import stream
import pandas as pd


In [3]:
#dataset = synth.RandomRBF(seed_model=2,n_classes=2,n_features=2,n_centroids=50)
ds = pd.read_csv('random.csv')
ds.head()
ds.dropna(axis=0,inplace=True)

y = ds.pop('.class')

In [4]:

fig, ax = plt.subplots(nrows=2,ncols=2,figsize = (10,10))



In [5]:
lst1 = []
lst2 = []
lst3 = []
i = 1
k,m = 0,0
tup = [(0,0),(0,1),(1,0),(1,1)]
l = 0
for x,y in stream.iter_pandas(ds,y):
    if i % 500 == 0 and i <= 2000: 
        k,m = tup[l]
        l += 1
        lst1 = np.array(lst1)
        lst2 = np.array(lst2) 
        lst3 = np.array(lst3) 
        ax[k,m].scatter(lst1[:,0],lst1[:,1])
        ax[k,m].scatter(lst2[:,0],lst2[:,1])
        ax[k,m].scatter(lst3[:,0],lst3[:,1])
        lst1 = []
        lst2 = []
        lst3 = []


    if y == 1.0 :
        lst1.append([x['X1'],x['X2']])
    elif y == 2.0 :
        lst2.append([x['X1'],x['X2']])
    else :
        lst3.append([x['X1'],x['X2']])

    i += 1 

In [6]:
fig.savefig('concept_drift.jpg')

In [ ]:
learner = CluStream(n_macro_clusters=6,max_micro_clusters=60,time_window=500)
test = drift.PageHinkley(100,delta=0.05,threshold=30,alpha= 0.9999,mode='both')



In [ ]:
def get_mean_dist(learner):
    ## Calcule la moyenne des max distances entres les centres des micro clusters
    mc_centers = [dict2numpy(x) for x in dict2numpy(learner.get_micro_clusters())]
    max_dist = np.max(distance_matrix(mc_centers,mc_centers,2),axis=1)
    return np.mean(max_dist)

In [ ]:
loop = 0
last_dist = 0
dist = 0
for x,y in dataset:
    if loop < 1000 :
        learner.learn_one(x)
        
    else :
        if loop == 1000 :
            print("Done training")
            learner.offline_cluster()

        if loop == 10000:
            break
        
        learner.learn_one(x)
        dist = get_mean_dist(learner)
        print(dist)
        test.update(dist )
        
        if test.drift_detected : 
            print(f"Drift Detected at instance: {loop}")
        
            
        
    loop += 1
    last_dist = dist

